## Raspagem de dados de escanteios da pagina de UM time no site TOTALCORNER

In [26]:
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cloudscraper
import copy

scraper = cloudscraper.create_scraper()

def get_goals_stats():
        options = uc.ChromeOptions() 
        options.headless = True 
        driver = uc.Chrome(use_subprocess=True, options=options)
        driver.get('https://www.soccerstats.com/matches.asp')
        html = driver.page_source
        driver.quit()
        soup = BeautifulSoup(html, 'html5lib')
        rows = soup.find_all("tr", bgcolor="#f0f0f0",height="22")
        tbody = soup.select("tbody tr.parent")
        leagues = {}
        for league in tbody:
                name = league.find_all('a',href=True, class_='vsmall')[0]['href'].split('=')[1]
                if len(league.find_all('font')) > 3:
                        goals = (league.find_all('font')[-1].get_text(strip=True).split(' ')[0].replace('(',''),
                                league.find_all('font')[-1].get_text(strip=True).split(' ')[3])
                        leagues[name] = goals
                else:
                        leagues[name] = (np.nan,np.nan)
        header = soup.find('tr',class_="child")
        header_list = [x.get_text(strip=True).replace('\xa0',' ') for x in header]
        header_list[0] = 'home_team'
        header_list.pop(1)
        header_list.pop(1)
        header_list.pop(-1)
        header_list.pop(-1)
        header_list += header_list
        header_list.append('league')
        header_list.append('league_avg_home_goals')
        header_list.append('league_avg_away_goals')
        games = []
        home = []
        away = []
        for row in rows:                
                stats = [x.get_text(strip=True) for x in row if x.text not in ['', 'stats']]
                league_name = row.find('a',href=True)
                while len(stats) > 13: 
                        stats.pop(1)                                        
                stats.pop(-1)
                if league_name:
                        stats.append(league_name['href'].split('=')[1].split('&')[0])
                if home == []:
                        home = copy.deepcopy(stats)
                elif away == []:
                        away = copy.deepcopy(stats)
                        if len(home) == 13:
                                league_name = home.pop(-1)
                                game = home + away
                                game.append(league_name)
                                game.append(leagues[league_name][0])
                                game.append(leagues[league_name][1])
                        else:
                                game = home + away
                        games.append(game)
                        home = []
                        away = []
        df = pd.DataFrame(games,columns=header_list)
        df.dropna(axis='index',inplace=True)        
        display(df)
        
        
get_goals_stats()

,home_team,GP,W%,FTS,CS,BTS,TG,GF,GA,1.5+,...,BTS,TG,GF,GA,1.5+,2.5+,3.5+,league,league_avg_home_goals,league_avg_away_goals
1,Annaba,13,54%,23%,54%,31%,2.15,1.38,0.77,46%,...,38%,1.69,0.46,1.23,54%,23%,8%,algeria2,1.38,0.90
2,El Eulma,13,54%,23%,46%,54%,2.08,1.23,0.85,54%,...,46%,2.38,1.08,1.31,77%,38%,23%,algeria2,1.38,0.90
3,El Harrach,13,54%,15%,46%,46%,2.38,1.54,0.85,69%,...,31%,1.77,0.77,1.00,46%,15%,8%,algeria2,1.38,0.90
4,Khemis El K.,13,46%,15%,54%,46%,2.08,1.31,0.77,69%,...,38%,2.00,0.54,1.46,54%,38%,15%,algeria2,1.38,0.90
5,Khroub,13,77%,0%,46%,54%,2.31,1.77,0.54,77%,...,62%,2.69,1.15,1.54,77%,46%,31%,algeria2,1.38,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Maroons,12,50%,17%,67%,33%,2.08,1.42,0.67,50%,...,50%,2.42,0.58,1.83,67%,58%,25%,uganda,1.24,0.79
165,UPDF,12,42%,33%,25%,50%,2.17,1.33,0.83,58%,...,38%,1.85,0.92,0.92,46%,23%,15%,uganda,1.24,0.79
166,Wakiso Giants,13,38%,23%,23%,62%,2.62,1.38,1.23,77%,...,33%,1.83,0.83,1.00,50%,33%,17%,uganda,1.24,0.79
167,Vipers,12,75%,17%,83%,17%,2.00,1.83,0.17,67%,...,58%,2.42,0.92,1.50,75%,42%,17%,uganda,1.24,0.79
